In [406]:
import math
import torch
import numpy as np

from sklearn.metrics import accuracy_score

In [258]:
class Variable():
    def __init__(self, name=None, ):
        if name[:2] != "x_":
            raise ValueError("Variable name must start with x_")
        self.name = name

    def gradient(self, values):
        grad = np.zeros(len(values))
        grad[int(self.name[2:]) - 1] = 1
        return grad
        
    def evaluate(self, values):
        # values is a dictionary
        return values[self.name]
    
    def __add__(self, other):
        return AdditionVariable(self, other)
    
    def __radd__(self, other):
        return AdditionVariable(self, other)
    
    def __sub__(self, other):
        return AdditionVariable(self, -1 * other)
    
    def __rsub__(self, other):
        return AdditionVariable(-1 * self, other)

    def __mul__(self, other):
        return MultiplicationVariable(self, other)
    
    def __rmul__(self, other):
        return MultiplicationVariable(self, other)
    
    def __pow__(self, other):
        return PowVariable(self, other)
    
    def __rpow__(self, other):
        return PowVariable(other, self)
    
    def __truediv__(self, other):
        return MultiplicationVariable(self, PowVariable(other, -1))
    
    def __rtruediv__(self, other):
        return MultiplicationVariable(PowVariable(self, -1), other)
    
    @staticmethod
    def exp(x):
        return ExpVariable(x)
    
    @staticmethod
    def log(x):
        return LogVariable(x)
    
    
class AdditionVariable(Variable):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    # override the evaluate method 
    def evaluate(self, values):
        if isinstance(self.right, (int, float)):
            return self.left.evaluate(values) + self.right
        return self.left.evaluate(values) + self.right.evaluate(values)
    
    def gradient(self, values):
        if isinstance(self.right, (float,int)):
            return self.left.gradient(values)
        if isinstance(self.left, (float,int)):
            return self.right.gradient(values)
        return self.left.gradient(values) + self.right.gradient(values)
    
    
class MultiplicationVariable(Variable):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    def evaluate(self, values):
        if isinstance(self.right, (int, float)):
            return self.left.evaluate(values) * self.right
        return self.left.evaluate(values) * self.right.evaluate(values)
    
    def gradient(self, values):
        if isinstance(self.right, (float,int)):
            return self.left.gradient(values) * self.right
        if isinstance(self.left, (float,int)):
            return self.right.gradient(values) * self.left
        return self.left.evaluate(values) * self.right.gradient(values) + self.left.gradient(values) * self.right.evaluate(values)
    
    
class PowVariable(Variable):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        
    def evaluate(self, values):
        if isinstance(self.right, (int, float)) and isinstance(self.left, (int, float)):
            return self.left ** self.right
        if isinstance(self.left, (int, float)):
            return self.left ** self.right.evaluate(values)
        if isinstance(self.right, (int, float)):
            return self.left.evaluate(values) ** self.right
        return self.left.evaluate(values) ** self.right.evaluate(values)
    
    def gradient(self, values):
        return self.right * self.left.evaluate(values)**(self.right - 1) * self.left.gradient(values)
    
    
class ExpVariable(Variable):
    def __init__(self, x, base=math.e):
        self.x = x
        self.base = base

    def evaluate(self, values):
        if isinstance(values, (int, float)):
            return self.base ** values
        return self.base ** self.x.evaluate(values)
    
    def gradient(self, values):
        return self.x.gradient(values) * math.e ** self.x.evaluate(values)
    
    
class LogVariable(Variable):
    def __init__(self, x):
        self.x = x
        
    def evaluate(self, values):
        if isinstance(self.x, (int, float)):
            return math.log(self.x)
        return math.log(self.x.evaluate(values))
    
    def gradient(self, values):
        if isinstance(self.x, (int, float)):
            return 1 / self.x
        return self.x.gradient(values) / self.x.evaluate(values)

59.97342844082839
[51.09815003 98.69630007 -1.5       ]


In [408]:
class LogisticRegression():
    def __init__(self, m=np.random.random(), b=np.random.random()):
        self.m = m
        self.b = b
        
        x_1 = Variable(name = "x_1")
        x_2 = Variable(name = "x_2")
        self.forward = lambda x: 1 / (1 + Variable.exp(-1 * (x_1 * x + x_2)))
    
    def loss(self, X, y):
        loss = 0
        for i in range(len(X)):
            forward_val = self.forward(X[i]).evaluate({"x_1": self.m, "x_2": self.b})
            loss += (y[i] * np.log(forward_val) + (1 - y[i]) * np.log(1 - forward_val))
        return -1 * loss
    
    def predict(self, X):
        answers = []
        for x in X:
            answers.append(self.forward(x).evaluate({"x_1": self.m, "x_2": self.b}))
        return answers
    
    def get_grad(self, X, y):
        grad = np.array([0, 0])
        for i in range(len(X)):
            grad = grad + (-1 * (y[i] * Variable.log(self.forward(X[i])) + (1 - y[i]) * Variable.log(1 - self.forward(X[i])))).gradient({'x_1': self.m, 'x_2': self.b})
        return grad / len(X)
    
    
    def fit(self, X, y, lr=0.1, epochs=50):
        for _ in range(epochs):
            
            grad = self.get_grad(X, y)
            # print(grad)
            print(self.loss(X, y))
            print(self.predict(X))
            print(accuracy_score(y, np.round(self.predict(X))))
            
            self.m -= lr * grad[0]
            self.b -= lr * grad[1]
            
            
        

In [351]:
# ------------------TESTING ZONE 👷🏻‍♀️🔧------------------

# Section 1: Testing Variable class
# Test 1: Dr. Z's Canvas one
x_1 = Variable(name = "x_1")
x_2 = Variable(name = "x_2")
x_3 = Variable(name = "x_3")

z = Variable.exp(x_1 + x_2**2) + 3 * Variable.log(27 - x_1 * x_2 * x_3)

print(z.evaluate({"x_1": 3, "x_2": 1, "x_3": 7})) # 59.97342844082839 ✅
print(z.gradient({"x_1": 3, "x_2": 1, "x_3": 7})) # [51.09815003 98.69630007 -1.5] ✅


59.97342844082839
[51.09815003 98.69630007 -1.5       ]


In [382]:
# Test 2: Unit tests
x_1 = Variable(name = "x_1")
x_2 = Variable(name = "x_2")

random = np.random.random(2) * 100

value_dict = {"x_1": random[0], "x_2": random[1]}

# 2.1: Test AdditionVariable
y_1 = x_1 + x_2
print(y_1.evaluate(value_dict) == random[0] + random[1]) # True ✅

y_1 = x_1 - x_2
print(y_1.evaluate(value_dict) == random[0] - random[1]) # True ✅

y_1 = -9.127 + x_2
print(y_1.evaluate(value_dict) == -9.127 + random[1]) # True ✅

# 2.2: Test MultiplicationVariable
y_2 = x_1 * x_2
print(y_2.evaluate(value_dict) == random[0] * random[1]) # True ✅

y_2 = x_1 / x_2
print(y_2.evaluate(value_dict) == random[0] / random[1]) # True ✅

y_2 = 9.127 * x_2
print(y_2.evaluate(value_dict) == 9.127 * random[1]) # True ✅

# 2.3: Test PowVariable
y_3 = x_1 ** x_2
print(y_3.evaluate(value_dict) == random[0] ** random[1]) # True ✅

y_3 = x_1 ** -9
print(y_3.evaluate(value_dict) == random[0] ** -9) # True ✅

y_3 = 9 ** x_2
print(y_3.evaluate(value_dict) == 9 ** random[1]) # True ✅

# 2.4: Test ExpVariable
y_4 = Variable.exp(x_1)
print(y_4.evaluate(value_dict) == math.e ** random[0]) # True ✅

# 2.5: Test LogVariable
y_5 = Variable.log(x_1)
print(y_5.evaluate(value_dict) == math.log(random[0])) # True ✅

True
True
True
True
True
True
True
True
True
True
True


In [385]:
# Test 3: A way to test gradients without doing all the ugly math...
#   I trust and adore everyone who has contributed to PyTorch

x_1 = Variable(name = "x_1")
x_2 = Variable(name = "x_2")
x_3 = Variable(name = "x_3")

z = Variable.exp(x_1 + x_2**2) + 3 * Variable.log(27 - x_1 * x_2 * x_3)

x_1t = torch.tensor(3., requires_grad=True)
x_2t = torch.tensor(1., requires_grad=True)
x_3t = torch.tensor(7., requires_grad=True)

zt = torch.exp(x_1t + x_2t**2) + 3 * torch.log(27 - x_1t * x_2t * x_3t)
zt.backward()
print(x_1t.grad) # 51.09815003 ✅
print(x_2t.grad) # 98.69630007 ✅
print(x_3t.grad) # -1.5 ✅

print(z.evaluate({"x_1": 3, "x_2": 1, "x_3": 7})) # 59.97342844082839 ✅
print(z.gradient({"x_1": 3, "x_2": 1, "x_3": 7})) # [51.09815003 98.69630007 -1.5] ✅


# NOTE TO SELF ADD SOME MORE CASES


tensor(51.0981)
tensor(98.6963)
tensor(-1.5000)
59.97342844082839
[51.09815003 98.69630007 -1.5       ]


In [410]:
# Section 2: Testing LogisticRegression class
model = LogisticRegression()
model.fit([0, 2, 3, 4, 5, 10, 32, 33, 35], [0, 0, 0, 0, 1, 1, 1, 1, 1], lr=0.3, epochs=100)

5.268692246876473
[0.5198576873615791, 0.6932051333952731, 0.7654838501870883, 0.8250322348655577, 0.8719886792200671, 0.9771980419504662, 0.9999928633343076, 0.9999950597751982, 0.9999976327254014]
0.5555555555555556
4.145206119486994
[0.49776189292771333, 0.5791030062093637, 0.618483070099972, 0.6563649916418562, 0.6923564916408474, 0.8363436940075324, 0.9947264448497378, 0.9955206387308816, 0.9967693309014503]
0.6666666666666666
4.06514827005369
[0.4822074808747767, 0.5302626350544598, 0.5541364745040691, 0.5777631268873659, 0.6010391813872594, 0.7090539921074452, 0.9528974881048704, 0.9570319685043801, 0.9642837293686998]
0.6666666666666666
4.128599266669295
[0.4711597126781822, 0.5857963080063334, 0.6405301937012721, 0.6918362253501883, 0.7388055646983304, 0.899802198982798, 0.9993100273932702, 0.99945229192071, 0.9996548974818525]
0.6666666666666666
3.8590399777634303
[0.4543687578254868, 0.5114262519310183, 0.5399363910676052, 0.5681875456467648, 0.5960019292000522, 0.7232623070